In [ ]:
import h5py
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

# Find the repo root (parent of current folder)
repo_root = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..')) if '__file__' not in globals() else os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
sys.path.append(repo_root)

# Add only immediate subfolders of the repo root to sys.path (much faster)
for item in os.listdir(repo_root):
    item_path = os.path.join(repo_root, item)
    if os.path.isdir(item_path):
        sys.path.append(item_path)

from read_scope_data import read_hdf5_scope_data, read_hdf5_scope_tarr, read_hdf5_all_scopes_channels, read_scope_channel_descriptions
from read_hdf5 import print_hdf5_info, print_scope_channel_descriptions
from process_xray_bdot_hdf5 import get_magnetron_power_data

%matplotlib qt

In [ ]:
ifn = r"F:\AUG2025\P24\00_He1kG430G_5800A_K-25_2025-08-12.hdf5"
print_hdf5_info(ifn)

In [ ]:
scope = 'xrayscope'
with h5py.File(ifn, 'r') as f:
    result = read_hdf5_all_scopes_channels(f, 1)
    descriptions = read_scope_channel_descriptions(f, scope)
    scp_data = result[scope]

In [ ]:
for ch, desc in scp_data['channels'].items():
    print(ch)

In [ ]:
P_data, tarr, I_data, V_data, Pref_data = get_magnetron_power_data(ifn, 1)
plt.figure()
plt.plot(tarr*1e6, P_data)

In [ ]:
print_scope_channel_descriptions(ifn, scope_name='bdotscope')

In [ ]:
scope_name = 'xrayscope'
channel_name = 'C2'
shot_number = 1
data = read_hdf5_scope_data(ifn, scope_name, channel_name, shot_number)
tarr = read_hdf5_scope_tarr(ifn, scope_name)

In [ ]:
plt.figure()
plt.plot(tarr*1e3, data)

In [ ]:
# Test the updated process_shot_bdot function
from process_xray_bdot_hdf5 import process_shot_bdot

# Test the function with the HDF5 file
stft_time, freq, stft_matrix1, stft_matrix2, stft_matrix3, channel_info = process_shot_bdot(ifn, shot_number=1, debug=True)

# Print channel information
print("\n=== Channel Information ===")
for channel, info in channel_info.items():
    field_comp = info['field_component']
    position = info['position']
    desc = info['description']
    print(f"{channel}: {field_comp} at {position} - {desc}")

# Check if data was successfully read
print(f"\nSTFT matrices availability:")
print(f"Matrix 1 (C1): {'Available' if stft_matrix1 is not None else 'None'}")
print(f"Matrix 2 (C2): {'Available' if stft_matrix2 is not None else 'None'}")
print(f"Matrix 3 (C3): {'Available' if stft_matrix3 is not None else 'None'}")

if stft_time is not None:
    print(f"STFT time array shape: {stft_time.shape}")
    print(f"Frequency array shape: {freq.shape}")